<a href="https://colab.research.google.com/github/lee-euijin/AppleFarm/blob/%EC%83%81%EC%A4%80/web_cam_capture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from tensorflow.keras import models

In [ ]:
# Loading model
from google.colab import drive
drive.mount('/content/drive')

model = models.load_model('/content/drive/My Drive/학교 수업 자료/딥러닝/Project/model.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install mediapipe opencv-python

     |████████████████████████████████| 36.1MB 113kB/s 


In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import mediapipe as mp # Face, Body, Hand Pose Detection 라이브러리

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  asassasasasa
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)  # 바이너리 파일 읽어옴
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# # function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
# def bbox_to_bytes(bbox_array):
#   """
#   Params:
#           bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
#   Returns:
#         bytes: Base64 image byte string
#   """
#   # convert array into PIL image
#   bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
#   iobuf = io.BytesIO()
#   # format bbox into png for return
#   bbox_PIL.save(iobuf, format='png')
#   # format return string
#   bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

#   return bbox_bytes


# def get_drawing_array(image_array): 
#     """
#     input: 
#           image_array: image array RGB size 512 x 512 from webcam

#     output: 
#           drawing_array: image RGBA size 512 x 512 only contain bounding box and text, 
#                               channel A value = 255 if the pixel contains drawing properties (lines, text) 
#                               else channel A value = 0
#     """
#     drawing_array = np.zeros([512,512,4], dtype=np.uint8)
#     img = letterbox(image_array, new_shape=opt.img_size)[0]

#     img = img.transpose(2, 0, 1)
#     img = np.ascontiguousarray(img)

#     img = torch.from_numpy(img).to(device)
#     img = img.float()  # uint8 to fp16/32
#     img /= 255.0  # (0 - 255) to (0.0 - 1.0)
#     if img.ndimension() == 3:
#         img = img.unsqueeze(0)

#     pred = model(img)[0]
#     # Apply NMS
#     pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)

#     # Process detections
#     det = pred[0]
#     if det is not None and len(det):
#         det[:, :4] = scale_coords(img.shape[2:], det[:, :4], image_array.shape).round()

#         # Write results
#         for *xyxy, conf, cls in det:
#             label = '%s %.2f' % (names[int(cls)], conf)
#             plot_one_box(xyxy, drawing_array, label=label, color=colors[int(cls)])

#     drawing_array[:,:,3] = (drawing_array.max(axis = 2) > 0 ).astype(int) * 255

#     return drawing_array

In [ ]:
# initialize the Haar Cascade face detection model
#face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) { // 꺼진게 아니면 
        window.requestAnimationFrame(onAnimationFrame);  //  함수는 기본적으로는 1초에 60번, 보통은 모니터에 주사율에 맞추어 함수를 실행함.
      }
      if (pendingResolve) { // 보류중
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480); // 이미지를 그려
          result = captureCanvas.toDataURL('image/jpeg', 0.8)  // 캔버스에 그린 그림을 문자열 형태로.  0.8 = encoderOptions

        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});  //모바일 장치의 후면 카메라를 요청하기 위한 코드:
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox)) # 자바스크립트 함수 사용하기 
  
  return data

In [45]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 1
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])    
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

      # Recolor Feed
      image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      # Make Detections
      results = holistic.process(image)
      # print(results.face_landmarks)
      
      # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
      
      # Recolor image back to BGR for rendering
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      
      # Draw face landmarks
      #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
      
      # landmark만 그릴 image
      landmark_img = np.zeros(image.shape, np.uint8)
      
      # Right hand
      mp_drawing.draw_landmarks(landmark_img, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      # Left Hand
      #mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      # Pose Detections
      #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

      '''
      # 웹캠에서 얻은 data를 train data로 추가 저장
      save_path = '/content/drive/My Drive/10/10_down/'

      if np.max(landmark_img) != 0:
        cv2.imwrite(save_path + 'frame_00_10_{:04d}.png'.format(count), landmark_img) 
        count += 1
      '''

      # Test data (= Captured from webcam)
      landmark_img = cv2.cvtColor(landmark_img, cv2.COLOR_RGB2GRAY)
      landmark_img = cv2.resize(landmark_img, (128,128))
      X_test = np.array(landmark_img)

      # Making prediction on test data
      X_test = X_test.reshape(1, 128, 128, 1)
      prediction = model.predict_proba(X_test)
      if np.max(prediction) < 0.2:
        print('none')
      else:
        print(prediction)
        print(np.argmax(prediction)) # prediction 결과
      
      
      

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


none
none
none
[[1.14729019e-07 3.54677672e-03 7.48946052e-03 8.32330167e-01
  1.51952356e-02 8.45769653e-04 1.39860036e-02 1.51159149e-03
  1.05334856e-01 1.97600257e-02]]
3
[[2.1804018e-17 2.1203391e-06 9.9998069e-01 1.2873014e-11 7.9691655e-16
  2.2371898e-09 1.7168075e-05 6.5140005e-09 1.1435168e-09 1.2669290e-13]]
2
[[2.6386149e-19 1.2430949e-07 9.9999928e-01 1.7362349e-15 7.7316293e-19
  1.0553655e-12 5.9019874e-07 3.4747596e-10 4.3617109e-13 3.5767608e-18]]
2
[[2.8661166e-17 2.6911580e-06 9.9999583e-01 6.7444897e-14 1.4259148e-20
  6.4544138e-16 2.2996587e-09 1.5146746e-06 6.6265222e-12 5.5487976e-14]]
2
[[1.3387233e-16 4.5911976e-05 9.9994588e-01 3.8413400e-10 1.1632842e-17
  7.9571153e-13 2.9525373e-09 8.2227771e-06 1.8661739e-09 1.5757303e-11]]
2
[[1.7705414e-18 9.3364797e-06 9.9999022e-01 1.6431688e-13 1.4072401e-23
  1.0259513e-13 5.1704783e-11 4.4861235e-07 2.6690984e-15 5.0953288e-15]]
2
[[7.26712595e-14 3.34087573e-02 9.66490090e-01 1.45078383e-09
  2.53529678e-20 1.3323